<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br><b>Image display demo for the IAU General Assembly LSST RSP workshop, August 2024</b> <br>
Rubin Science Platform deployment: data.lsst.cloud <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Last verified to run: Tue Aug 13 2024 <br>
Contact author: Christina Williams <br>




<br>


Learn more about the Data Preview 0 data set (e.g., image types, catalog tables)
and the LSST Science Pipelines and Rubin Science Platform functionality using the
documentation and resources available at <a href="https://dp0.lsst.io/">dp0.lsst.io</a>.

<br>


**Description:** Learn to display and manipulate images using the LSST Science Pipelines.

**Skills:** Display and manipulate images, explore image mask planes, and create cutouts.

**LSST Data Products:** Images (calexp and deepCoadd image types), table that records the location and the type of image (ivoa.ObsCore table)

**Packages:** lsst.afw.display, lsst.daf.butler, lsst.geom, lsst.afw.image

**Credit:** This tutorial is based on multiple tutorials developed by many people, including tutorial notebooks DP02_01, DP02_02a,b,c, and DP02_03a,b that are published by the Rubin Community Science Team [at the DP0.2 Tutorial-Notebooks github repository](https://pipelines.lsst.io/getting-started/display.html) Rendered versions (including output) are available [here](https://dp0-2.lsst.io/tutorials-examples/rendered-tutorial-notebooks.html).

More examples of the use of `lsst.afw.display` can be found in the [LSST Science Pipelines documentation](https://pipelines.lsst.io/getting-started/display.html).

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>.
Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This tutorial is designed to introduce users to accessing imaging data products produced by the LSST pipelines, their format, and basic image display and manipulation. 

It also introduces LSST pipeline packages that are useful for basic access:  the `lsst.afw.display` library that enables the visual inspection of image data. The [`lsst.afw` library](https://github.com/lsst/afw) provides an "Astronomical Framework" (afw) while the `lsst.daf.*` libraries (see, e.g., [daf_base](https://github.com/lsst/daf_base)) provide a "Data Access Framework" (daf). Both libraries are used in this tutorial, with the `lsst.daf.butler` library used to access image data and the `lsst.afw.display` library used to display the image data.

### 1.1. Package imports

Below, the `matplotlib.pyplot` sublibrary is imported for plotting, and the [`astropy.wcs`](https://docs.astropy.org/en/stable/wcs/index.html) package is imported for dealing with World Coordinate Systems (WCS).
The [`matplotlib`](https://matplotlib.org/) and [`astropy`](http://www.astropy.org/) libraries are widely used Python libraries for plotting, scientific computing, and astronomical data analysis.
The `gc` (garbage collector) package is imported to help clear the memory of large plots.

The `lsst.afw.display` and `lsst.afw.image` modules are loaded to gain access to the image visualization routines we'd like to use, and the `lsst.daf.butler` and `lsst.rsp.get_tap_service` modules are used to access data products.
The `lsst.geom` package is used for dealing with sky coordinates.

In [ ]:
# general-use python packages
import matplotlib.pyplot as plt
import gc
import sys

# Astropy packages
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.wcs import WCS

# LSST Pipeline packages for data access:
from lsst.daf.butler import Butler
from lsst.rsp import get_tap_service

# LSST Pipeline package for dealing with sky coordinates
import lsst.geom as geom

# LSST Pipieline packages for image manipulation and display
import lsst.afw.display as afwDisplay
from lsst.afw.image import Exposure, ExposureF


### 1.2. Define functions and parameters

#### 1.2.1. Functions

Matplotlib stores the data array associated with an image that is plotted. Since the LSST Charge-Coupled Device (CCD) detector images are large (~4k x 4k pixels), this can eventually lead to a memory overflow, which will cause the notebook kernel to die. To mitigate this issue, we define a function to clean up after we plot them.

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

Set the parameters of `matplotlib.pyplot` to give us a large default size for an image, and set some other parameters to make the default plot style be color-blind friendly and otherwise make plots look nice.

In [ ]:
plt.style.use('tableau-colorblind10')
%matplotlib inline

In [ ]:
params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [8, 8],
          'figure.facecolor': 'White'
          }
plt.rcParams.update(params)

## 2. Intro to Image Data

This tutorial focuses on two common types of images people will interact with: calexps and deepCoadds.

<b>calexp: A calibrated exposure (single image in a single filter).</b>

Calexp files are uniquely identified by visit (when observation was taken) and detector in the camera.  

<img src="figures/img_demonstration.png" width="400">
Figure 1: Example of many calexp footprints (magenta) that are combined into deepCoadds (gray)
<br><br>
<b>deepCoadd: A combination of single images into a deep stack or Coadd.</b>

The LSST Science Pipelines processes and stores deepCoadd images in tracts and patches.

tract: A portion of sky within the LSST all-sky tessellation (sky map); divided into patches.

patch: A quadrilateral sub-region of a tract, of a size that fits easily into memory on desktop computers.

<img src="figures/dp02_patch_tract.png" width="700">
Figure 2: Example tract divided into patches in DP0.2 (Abolfathi et al. 2021)
<br><br>

To retrieve and display an image at a desired coordinate, users have to specify their image type, and additional parameters, which make up whats called a `dataId` : tract, patch and filter are needed in the case of `deepCoadd` ; only the visit and detector are needed in the case of `calexp`.

## 2.1. Retrieve an image using the LSST Butler

The butler (<a href="https://pipelines.lsst.io/modules/lsst.daf.butler/index.html">documentation</a>) is an LSST Science Pipelines software package to fetch LSST data without having to know its location or format. This is the preferred method for image retrieval using the RSP Notebook aspect.

The DP0.2 data set contains simulated images from the LSST DESC Data Challenge 2 (DC2) that have been reprocessed by the LSST Project using a more recent version of the LSST Science Pipelines.  

### 2.1.1 Create an instance of the butler 
Specify data set of imaging contained in the RSP to use. Request simulated DP0.2 imaging by directing the Butler to the `dp02` data repository configuration, and the collection `2.2i/runs/DP0.2` specifies which processing run using the LSST pipelines to use. 

It will return an informative statement about credentials being found.

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

### 2.1.2 Identify and retrieve a deepCoadd
There is a cool-looking galaxy cluster in the DP0.2 simulated data at RA = 03h42m59.0s, Dec = -32d16m09s (in degrees, 55.745834, -32.269167).

Use the LSST `geom` package to define a `SpherePoint` (an LSST pipeline object to hold coordinates) for the cluster's coordinates (<a href="https://pipelines.lsst.io/modules/lsst.geom/index.html">lsst.geom documentation</a>).


In [ ]:
my_ra = 55.745834
my_dec = -32.269167

my_spherePoint = geom.SpherePoint(my_ra*geom.degrees,
                                  my_dec*geom.degrees)
print(my_spherePoint)

Retrieve the DP0.2 sky map from the butler. This is simply a reference of how the all sky area is divided up into patch and tracts. Use it to identify the tract and patch for the cluster's coordinates (<a href="http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/skymap.html">skymap documentation</a>).

In [ ]:
skymap = butler.get('skyMap')

tract = skymap.findTract(my_spherePoint)
patch = tract.findPatch(my_spherePoint)

my_tract = tract.tract_id
my_patch = patch.getSequentialIndex()

print('my_tract: ', my_tract)
print('my_patch: ', my_patch)



Use the butler to retrieve the i-band deepCoadd for the tract and patch. To do this, we define a dictionary called `dataId` with the required information (for deepCoadds, to uniquely identify the image, the user must provide 3 things: the filter (band), patch, and tract). 

In [ ]:
dataId = {'band': 'i', 'tract': my_tract, 'patch': my_patch}
my_deepCoadd = butler.get('deepCoadd', dataId=dataId)


### 2.1.3 LSST image format

The `my_deepCoadd` returned by the Butler contains the science image, and also has a number of extensions containing other data or information about the image. This includes the science image, a mask plane containing the masked pixels and why they were masked, and spatially varying PSF information.

These examples will focus on just the science and mask extensions (see the tutorial-notebooks github repo for additional demonstrations). 

The extension containing the science image and masks can be returned using the `image` and `mask` attributes. One can retrieve just an array containing the pixels values using `.array` attribute.

In [ ]:
deepCoadd_image = my_deepCoadd.image
deepCoadd_pixels = my_deepCoadd.image.array
deepCoadd_mask = my_deepCoadd.mask
deepCoadd_var = my_deepCoadd.variance


## 3. Basic image visualization

### 3.1 Use AFWDisplay to visualize the image
Image data retrieved with the butler is an `Exposure` or `ExposureF` (F=float) Python object. Exposures are the native format for imaging data taken with Rubin and processed by LSST Pipelines. They are powerful representations of image data because they contain not only the image data, but also a variance image for uncertainty propagation, a bit mask image, and key-value metadata. In the next section, we will use `afwDisplay` to visualize the image and mask associated with this Exposure. More documentation on accessing and visualizing an Exposure object be found [here](https://pipelines.lsst.io/getting-started/display.html).

The image contained in the `ExposureF` object can be displayed several different ways. A simple option is to use the LSST Science Pipelines package `afwDisplay`. There is some <a href="https://pipelines.lsst.io/modules/lsst.afw.display/index.html">documentation for afwDisplay</a> available, and other DP0 tutorials LINK go into more detail about all the display options (e.g., overlaying mask data to show bad pixels).



#### 3.1.1 Interactive Display using afwDisplay

First, explore the image using an interactive display called `firefly`, which is the RSP's interface for interactive image display and manipulation. 

1. Create an alias to the `lsst.afw.display.Display` method that will allow us to display the data to the screen.  This alias will be called `firefly_display`.
2. Display the image by providing the `mtv()` method the `deepCoadd.image` member of our calibrated image retrieved by the `butler`.

In [ ]:
afwDisplay.setDefaultBackend('firefly')
firefly_display = afwDisplay.Display(frame=1)
firefly_display.mtv(deepCoadd_image)


#### 3.1.2 Static Image Display using afwDisplay

It is also possible to plot statically in the notebook. To do this, set the backend of afwDisplay to matplotlib.


In [ ]:
afwDisplay.setDefaultBackend('matplotlib')


The following code cell creates a matplotlib.pyplot figure; aliases `lsst.afw.display.Display` as `display`; 
sets the image stretch for the pixel shading (set by the algorithm asinh -- familiar from SDSS images -- with a range of values set by zscale); displays the image data using the `mtv` method; and turns on the x and y axes labels (pixel coordinates).

At the end, remove the figure data to free some memory.


In [ ]:
fig = plt.figure(figsize=(10, 8))
display = afwDisplay.Display(1)
display.scale('asinh', 'zscale')
display.mtv(my_deepCoadd.image)
plt.gca().axis('on')
plt.show()

remove_figure(fig)

#### 3.1.3 More information and help

To find more information, the next cell will print the `lsst.afw.display` methods that are available for this display type to the screen. 

In [ ]:
method_list = [fun for fun in dir(display) if callable(getattr(display, fun))]
print(method_list)

To learn more about the afwDisplay package and its tasks (e.g. `mtv`), use the help function.

In [ ]:
help(display.mtv)


### 3.2 Working with World Coordinate Systems (WCS)

To see the image axes in sky coordinates instead of pixel coordinates, a simple option is to use astropy's World Coordinate System (WCS) package, along with matplotlib.pyplot's `subplot`, `imshow`, and `grid` functions. 
Recall that we imported `matplotlib.pyplot` as `plt` already, and that we imported the `astropy.wcs.WCS` constructor as simply `WCS`.
Find more information about [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) and [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) (`cmap`).

To do this, we:
1. Set the figure's projection to be the WCS of the `deepCoadd`.
2. Define the extent in pixel coordinates using the bounding box.
3. Display the deepCoadd image data array using the gray colormap (cmap)
4. Add solid white grid lines.
5. Label the axes, and show the plot.
6. Remove the underlying data from memory.

In [ ]:
fig = plt.figure()

my_deepCoadd_WCS = WCS(my_deepCoadd.getWcs().getFitsMetadata())

plt.subplot(projection = my_deepCoadd_WCS)

my_deepCoadd_extent = (my_deepCoadd.getBBox().beginX, my_deepCoadd.getBBox().endX,
                       my_deepCoadd.getBBox().beginY, my_deepCoadd.getBBox().endY)

im = plt.imshow(my_deepCoadd.image.array, cmap='gray', vmin=-2.0, vmax=4,
                extent=my_deepCoadd_extent, origin='lower')

plt.grid(color='white', ls='solid')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
plt.show()
remove_figure(fig)

**Congratulations!** We've plotted an image in various ways using `lsst.afw.display`!

### 3.3. Use afwDisplay to visualize the image and mask plane

The `deepCoadd` returned by the butler contains more than just the image pixel values (see this [image tutorial](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/Basics/Calexp_guided_tour.ipynb) for more details). 
One other component is the mask associated with the image. 
A mask is composed of a set of "mask planes," 2D binary bit maps corresponding to pixels that are masked for various reasons (see [here](https://pipelines.lsst.io/v/DM-11392/getting-started/display.html#interpreting-displayed-mask-colors) for more details).

The `afwDisplay` package maps each bit in the mask plane to a specific display color. 
We can view this mapping using the code in the following cell. 
We can also use the `setMaskPlaneColor` method to change the colors that `afwDisplay` uses for each mask plane.

Print the colors associated to each plane in the mask.

In [ ]:
print("Mask plane bit definitions:\n", display.getMaskPlaneColor())
print("\nMask plane methods:\n")
help(display.setMaskPlaneColor)

Let's plot the image and mask side-by-side using matplotlib.

Use `plt.sca(ax[0])` to set the first axis as current, and then `plt.sca(ax[1])` to switch to the second axis.
Using `plt.tight_layout()` with multi-axis figures helps to avoid axis overlap or excessive white spaces and results in a nicer-looking plot.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 7))

# left panel
plt.sca(ax[0])
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(my_deepCoadd.image)

# right panel
plt.sca(ax[1])
display2 = afwDisplay.Display(frame=fig)
display2.mtv(my_deepCoadd.mask)
plt.tight_layout()

plt.show()

remove_figure(fig)

The `afwDisplay` package also provides a nice interface for plotting the mask on top of the image using the `my_deepCoadd.maskedImage`. 
The mask will also be plotted on top of the image if you pass the `my_deepCoadd` itself to `mtv` (as is done later in this notebook).

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)

for maskName, maskBit in my_deepCoadd.mask.getMaskPlaneDict().items(): 
    print(f'{maskName:18s} (bit=2**{maskBit:02d})')
    display.setMaskTransparency(100,maskName)

display.setMaskTransparency(10,'DETECTED') 
#display.setMaskPlaneColor('SENSOR_EDGE', 'yellow')


display.scale('linear', 'zscale')
display.mtv(my_deepCoadd.maskedImage)
plt.show()
remove_figure(fig)

To investigate the mask in a bit more detail, we'll follow the same steps as above to display the image, but we'll add a few modifications

1. We explicitly set the transparency of the overplotted mask
   (as a percentage: 0 = opaque, 100 = transparent)
2. We explicitly set the color of the 'DETECTED' mask plane to 'blue' (i.e., all pixels associated with detected objects).
3. We pass the full `my_deepCoadd` object to `mtv` instead of just the image plane.

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.setMaskTransparency(80)
display.setMaskPlaneColor('DETECTED', 'green')
display.mtv(my_deepCoadd)
plt.show()

remove_figure(fig)


The image above is displaying pixel coordinates (note that the coadd patch is part of a larger coadd image called a "tract", so the pixel values do not start at 0,0), but in general it is more useful to be able to select a region based on RA, Dec coordinates. To do this, we'll use the world coordinate system (WCS) object associated with the image.

Extract the WCS solution, which provides the mapping between XY pixel values and sky coordinates, and print it.

In [ ]:
wcs = my_deepCoadd.getWcs()
print(wcs)

The cluster seems to be centered at about (X, Y) ~ (12500, 8500).
Use the "pixelToSky" method of the WCS to get the sky coordinates.

In [ ]:
radec = wcs.pixelToSky(12500, 8500)
cluster_ra, cluster_dec = radec.getRa().asDegrees(), radec.getDec().asDegrees()
print(cluster_ra, cluster_dec)


## 4. Accessing Images via TAP query: The ObsCore table

It is also possible to use TAP queries to access imaging information from an LSST catalog, and does not require the use of LSST Pipelines like the Butler does. A use case for this method of image access is if accessing from outside the RSP.

The LSST `ObsCore` table contains information about the images
stored in the LSST's [data butler](https://pipelines.lsst.io/modules/lsst.daf.butler/index.html) registry (for more information, see data management document, [DMTN-236](https://dmtn-236.lsst.io/)). The IVOA in the catalog's title `ivoa.ObsCore` stands for International Virtual Observatory Alliance, which sets standards for data storage (the Virtual Observatory; VO) and Rubin's data products are consistent with these VO standards.

For this part of the tutorial, import some standard VO packages that will enable data access via VO standards.

In [ ]:
# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# more LSST Pipeline packages
from lsst.resources import ResourcePath
from lsst.afw.fits import MemFileManager
from lsst.afw.image import ExposureF


Like Tutorial 2 on catalogs, we initialize the TAP service to allow queries to the `ivoa.ObsCore` table. The second line we will create store the authentication, which will be needed to access the imaging (if proprietary) from outside the RSP environment (for more information see https://rsp.lsst.io/guides/auth/creating-user-tokens.html). 

In [ ]:
service = get_tap_service("tap")
auth_session = service._session

Retrieve the names of the columns and their data types, descriptions and units,
and display these properties as a pandas table (allows display of all rows/columns). The output is a list of the image metadata that is available in the `ObsCore` table.

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'ivoa.ObsCore'"

results = service.search(query)
results.to_table().to_pandas()

Useful metadata includes `s_region`, which is the bounding box (spatial extent) of the image. This can be used to identify which images cover the ra/dec of interest.

The following query selects all columns from the `ObsCore` table
for all images that contain the coordinate for the galaxy cluster within the `s_region`. (In other words, where the statment that the `s_region` contains 
the ra & dec of the point is `True` or `= 1`.)

For large queries like this one, we submit "asynchronously" and wait for the job to complete before continuing.

In [ ]:
query = "SELECT * FROM ivoa.ObsCore "\
        "WHERE CONTAINS(POINT('ICRS', " + str(my_ra) + \
        ", " + str(my_dec) + "), s_region) = 1"

# Define the service job using the query, then run it.
job = service.submit_job(query)
job.run()

Wait until the job is completed.

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

When the job phase is "COMPLETED", fetch (retrieve) the results,
and print the number of rows returned.
This is the number of images found that overlap the specified point.

In [ ]:
results = job.fetch_result().to_table().to_pandas()
print(len(results))


Retrive image data (headers and pixel data), for imaging. This time, we will retrieve a `calexp` image (which are single visit exposures on sky; see Section 2). 

`calexp` (calibrated exposures) are classified as calibration level (`calib_level`) of 2. `calib_level` is an IVOA convention to classify the different level of processing that has been applied to the observed dataset. 


In the RSP, the dataset calib_level numbers correspond to:

    1. Raw images (not science-ready)
    2. Calibrated single-visit exposures (calexp)
    3. Coadded images and difference images (e.g. deepCoadd)


For Rubin/LSST, we will down-sample the query to only return `calexp` images (i.e. `calib_level`=2) taken in the i-band. 


In [ ]:
query = "SELECT * FROM ivoa.ObsCore "\
        "WHERE CONTAINS(POINT('ICRS', " + \
        str(my_ra) + ", " + str(my_dec) + "), s_region) = 1 "\
        "AND lsst_band = 'i' "\
        "AND calib_level = 2"
print(query)

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
results = job.fetch_result()
print(len(results))



Option to view results as an `astropy` table.

In [ ]:
# results.to_table()

This ra/dec is covered by 151 `calexp` images.  For this demonstration only one of the 151 images is needed. So, extract any row into `results_0` (in this case we will use row 100).

In [ ]:
results_0 = results[100]

Print selected metadata:
 * dataproduct subtype
 * CCD visit identifier (`lsst_ccdvisitid`)
 * visit identifier and detector number
 * central RA and Dec
 * band (filter)
 * exposure start time `t_min` (a modified julian date, MJD)
 * `s_region` (defines the corner coordinates of the image; i.e. its extent)
 * a unique `obs_id` in the TAP service

In [ ]:
print(results_0['dataproduct_subtype'])
print(results_0['lsst_ccdvisitid'])
print("visit number = ", results_0['lsst_visit'], 
      "detector number = ", results_0['lsst_detector'])
print('%7.4f %8.4f' % (results_0['s_ra'], results_0['s_dec']))
print(results_0['lsst_band'])
print(results_0['t_min'])
print(results_0['s_region'])
print(results_0['obs_id'])

#### 4.1 Access the image via a Datalink URL

Data is accessible via an `access_url` and the format of the
data is stored in `access_format`.

In [ ]:
print(results_0['access_url'])
print(results_0['access_format'])

Above, the format indicates that the image information is stored as a `votable` which is accessible via Datalink.

Obtain the table `dl_results` using Datalink and the `auth_session` that was defined earlier to authenticate data access.

In [ ]:

dl_results = DatalinkResults.from_result_url(results_0['access_url'],
                                             session=auth_session)

Option to view `dl_results` as an astropy table.

In [ ]:
# dl_results.to_table()

For security reasons the `dl_results` table holds a *different* `access_url` that
provides direct access to the image - define it as `image_url`
and print it to see how it is different from the `access_url` above. We need the datalink URL, which was provided once authenticated, to access the image.

In [ ]:
image_url = dl_results['access_url'][0]
print(image_url)

Currently, since the LSST pipelines are still in development, there is not currently  a way to read the ExposureF directly from a url right now. To access the image, first do the extra step of allocating memory for the image data before displaying the image.


In [ ]:
fits_data = ResourcePath(image_url).read()

mem = MemFileManager(len(fits_data))
mem.setData(fits_data, len(fits_data))
exposure = ExposureF(mem)


Display the image

In [ ]:

fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(exposure.image)
plt.show()


### 4.3 Optional: Creating cutouts of LSST images using the cutout tool

These pipelines are still under development, in the future its likely these will be packaged into more user-fiendly functions. We may develop new ways that would be recommended ways for users to do these operations.

Image cutouts are an important tool that enables a faster way to visually inspect science targets. Below we demonstrate the current way to perform image cutouts, using the IVOA-compatible cutout tool.

TBD: Below using deepCOadds, swap out for the calexp?



In [ ]:

#spherePoint = geom.SpherePoint(my_ra*geom.degrees, my_dec*geom.degrees)
spherePoint = my_spherePoint
# perform the search (maybe use calexp search from above??
query = "SELECT access_format, access_url, dataproduct_subtype, " + \
    "lsst_patch, lsst_tract, lsst_band, s_ra, s_dec  " + \
    "FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
    "AND obs_collection = 'LSST.DP02' " + \
    "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
    "AND lsst_tract = " + str(my_tract) + " " + \
    "AND lsst_patch = " + str(my_patch) + " " + \
    "AND lsst_band = 'i' "

#results = service.search(query)
#results.to_table().show_in_notebook()

# obtain the datalink for the data in the LSST database, as we did for the calexp 
# (can we comment this stuff out cause we did it above?)
dataLinkUrl = results[0].getdataurl()

#auth_session = service._session
#dl_results = DatalinkResults.from_result_url(dataLinkUrl,
                                            # session=auth_session)

f"Datalink status: {dl_results.status}. Datalink service url: {dataLinkUrl}"

sq = SodaQuery.from_resource(dl_results,
                             dl_results.get_adhocservice_by_id("cutout-sync"),
                             session=auth_session)

# build the shape of the cutout. Circle returns a square cutout with 
# edge size = diameter of the subtended circle
sphereRadius = 0.05 * u.deg
sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
             spherePoint.getDec().asDegrees() * u.deg,
             sphereRadius)

# store the cutout data in memory:
cutout_bytes = sq.execute_stream().read()
mem = MemFileManager(len(cutout_bytes))
mem.setData(cutout_bytes, len(cutout_bytes))
exposure = ExposureF(mem)

# plot the image:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(exposure.image)
plt.show()




## Additional Documentation

If you'd like some more information on `lsst.afw.display`, please have a look at the following websites:

* [Info on image indexing conventions.](https://pipelines.lsst.io/modules/lsst.afw.image/indexing-conventions.html)  
* [afw.display Doxygen website](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1display.html)  
* [afw.display GitHub website](https://github.com/RobertLuptonTheGood/afw/tree/master/python/lsst/afw/display)  
* [Getting Started on Image Display (pipelines.lsst.io)](https://pipelines.lsst.io/getting-started/display.html)